### HYPOTHESIS
hipotesis_1 = "Hay mucha demanda de ayudas y servicios sociales en determinados distritos (valorar si es en todos) y poco personal para absorber esta demanda"

hipotesis_2 = "Valorar si hay suficientes recursos no humanos para atender las demandas de los usuarios"

In [2]:
"""
IMPORTS
"""
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import openpyxl


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

### 1.- Data Collecting

La fuente principal de todos los datos obtenidos ha sido a través de las páginas del ayuntamiento de Madrid (MUNIMADRID y PORTAL DE DATOS ABIERTOS DEL AYUNTAMIENTO DE MADRID)

In [ ]:
# guarda en variables los datasets y su fuente

# df_demandas = pd.DataFrame()
# URL_demandas_de_servicio="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=40ab4e8b0327d610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
# Solicitudes de procesos solicitados por los usuarios

# df_urgencias = pd.DataFrame()
# URL_urgencias_atendidas="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=0b006dace9578610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
# URGENCIAS ATENDIDAS EN LOS CENTROS. ESTAS URGENCIAS SON DE LA SIGUIENTES TIPOLOGIAS 
# Malos tratos                                      --      Código: 10
# Abandono                                          --      Código: 20
# Alojamiento/Vivienda                              --      Código: 30
# Desprotección social problema salud               --      Código: 40
# Desprotección socio-familiar                      --      Código: 50

# df_centros_sociales = pd.DataFrame()
# URL_centros_sociales="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=7c123e08dafc8410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
# Listado con geoposicionamiento de los diferentes centros sociales que se encuentran en Madrid, tanto del ayuntamiento como otros que no son del ayuntamiento (por ejemplo el Centro de Atención Social del "Casino de la Reina").


# OTROS POSIBLES DF QUE PUDIERA NECESITAR:
#URL_datos_abiertos_madrid="https://datos.madrid.es/portal/site/egob/menuitem.9e1e2f6404558187cf35cf3584f1a5a0/?vgnextoid=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
#URL_ayudas_sociales="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=00937b777f8ea610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"
#URL_ranking_vulnerabilidad="https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=d029ed1e80d38610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"


In [5]:
def regex_replacement(df, columna):
    '''
    Función para elimina los espacios en blanco y sustituye los rangos de edad.
    '''
    
    df[columna] = df[columna].str.replace(' - ','-')
    df[columna] = df[columna].str.replace('- ','-')
    
    df[columna] = df[columna].str.replace("\A0-3", "<=18")
    df[columna] = df[columna].str.replace("\A4-11", "<=18")
    df[columna] = df[columna].str.replace("\A12-17", "<=18")
    df[columna] = df[columna].str.replace("\A18-24", "19-29")
    df[columna] = df[columna].str.replace("\A25-29", "19-29")
    df[columna] = df[columna].str.replace("\A30-34", "30-39")
    df[columna] = df[columna].str.replace("\A35-39", "30-39")
    df[columna] = df[columna].str.replace("\A40-44", "40-49")
    df[columna] = df[columna].str.replace("\A44-49", "40-49")
    df[columna] = df[columna].str.replace("\A45-49", "40-49")
    df[columna] = df[columna].str.replace("\A50-54", "50-59")
    df[columna] = df[columna].str.replace("\A55-59", "50-59")
    df[columna] = df[columna].str.replace("\A60-64", "60-64")
    df[columna] = df[columna].str.replace("\A65-69", ">=65")
    df[columna] = df[columna].str.replace("\A75-79", ">=65")
    df[columna] = df[columna].str.replace("\A80-84", ">=65")
    df[columna] = df[columna].str.replace("\A70-74", ">=65")
    df[columna] = df[columna].str.replace("\A>=85", ">=65")


    return df

### 2.- Loading Datasets in data\Outcomes

Vamos a tomar los datos de los diferentes Datasets desde 2018 hasta 2021. 

In [3]:
df_2021=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2021.csv",sep=';',encoding='ISO-8859-1',index_col='Secuencia')   
#Este  dataset solo contine información hasta julio de 2021
df_2020=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2020.csv",sep=';',encoding='utf-8',index_col='Secuencia')

df_2019=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2019.csv",sep=';',encoding='ISO-8859-1',index_col='Secuencia')

df_2018=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2018.csv",sep=';',encoding='ISO-8859-1',index_col='Secuencia')

In [4]:
df_urgencias=pd.concat([df_2021,df_2020,df_2019,df_2018], ignore_index=True)
df_urgencias

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
0,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1103,SAN ISIDRO,060,25 - 29,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021
1,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,083,30 - 34,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,04/01/2021
2,74,CSS Santa Hortensia,5,CHAMARTÍN,05,CHAMARTÍN,0503,CIUDAD JARDIN,043,70 - 74,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021
3,91,CSS Luis Vives,15,CIUDAD LINEAL,15,CIUDAD LINEAL,1503,QUINTANA,098,25 - 29,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021
4,80,CSS Gallur,10,LATINA,10,LATINA,1003,LUCERO,068,>= 85,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37901,98,CSS Pablo Casals,20,SAN BLAS-CANILLEJAS,20,SAN BLAS-CANILLEJAS,2004,ARCOS,48,30 - 34,Española,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,28/12/2018
37902,85,CSS Loyola de Palacio,12,USERA,12,USERA,1203,SAN FERMIN,100,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,28/12/2018
37903,84,CSS San Filiberto,12,USERA,12,USERA,1205,MOSCARDO,53,35 - 39,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,28/12/2018
37904,84,CSS San Filiberto,12,USERA,12,USERA,1205,MOSCARDO,74,65 - 69,Española,H,1,Primera Atención,30.0,Alojamiento/Vivienda,28/12/2018


In [8]:
df_urgencias.to_excel(r'E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\resources\img\df_urgencias.xls',index=False,header=True)

c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """Entry point for launching an IPython kernel.


In [59]:
df_urgencias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37906 entries, 0 to 37905
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Código Centro                 37906 non-null  int64  
 1   Centro                        37906 non-null  object 
 2   Código Distrito Centro        37906 non-null  int64  
 3   Distrito Centro               37906 non-null  object 
 4   Código Distrito               37906 non-null  object 
 5   Distrito                      37906 non-null  object 
 6   Código Barrio                 37906 non-null  object 
 7   Barrio                        37906 non-null  object 
 8   Sección Censal                37906 non-null  object 
 9   Tramo Edad                    37906 non-null  object 
 10  Nacionalidad                  37906 non-null  object 
 11  Sexo                          37906 non-null  object 
 12  Código Tipo Atención          37906 non-null  int64  
 13  T

En este DataFrame se observa  que hay bastantes *datos nulos (NaN)* relacionados con la variable target. 

Por ello antes de continuar con el proceso vamos a realizar un rellenado de huecos Dataset por Dataset para intentar que el impacto final sea el menor posible.

### 3.- Featuring Engineering

El proceso que vamos a realizar es general para cada Dataset.

Aqui vamos a indicar para un único Dataset, pero se realizará en todos.


#### 2018

In [7]:
df_2018.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
Secuencia,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0202,ACACIAS,25,55 - 59,Española,M,1,Primera Atención,10.0,Malos Tratos,02/01/2018
2,83,CSS Zaida,11,CARABANCHEL,10,LATINA,1004,ALUCHE,82,50 - 54,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2018
3,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0102,EMBAJADORES,26,75 - 79,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2018
4,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0101,PALACIO,21,65 - 69,Española,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2018
5,74,CSS Santa Hortensia,5,CHAMARTÍN,5,CHAMARTÍN,0501,EL VISO,5,80 - 84,Española,H,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2018


Vamos a agrupar las __fechas de edades__ para tener un rango más manejable.

Así agruparemos < 18 años a todos los grupos que sean inferiores a esta edad.

Agruparemos >65 a todos los grupos que sean superiores a esta edad.

Agruparemos de 10 en 10 años al resto.

In [8]:
df_2018['Tramo Edad'].value_counts()

>= 85      1193
80 - 84     559
35 - 39     450
40 - 44     439
30 - 34     413
75 - 79     352
45- 49      342
50 - 54     312
70 - 74     287
18 - 24     284
25 - 29     271
55 - 59     244
65 - 69     224
60 - 64     214
12 - 17      20
             12
4 - 11       10
0 - 3         5
Name: Tramo Edad, dtype: int64

Vemos que hay __12 registros en blanco__ que no hemos sustituido.

Vamos a proceder a reemplazarlos por el registro anterior, ya que no afectará mucho a las previsiones del módelo.

In [9]:
df_2018['Tramo Edad']=df_2018['Tramo Edad'].mask(df_2018['Tramo Edad'] == ' ').fillna(method='ffill')#,inplace=True)
df_2018['Sexo']=df_2018['Sexo'].mask(df_2018['Sexo'] == ' ').fillna(method='ffill')
df_2018['Nacionalidad']=df_2018['Nacionalidad'].mask(df_2018['Nacionalidad'] == ' ').fillna(method='ffill')
regex_replacement(df_2018, 'Tramo Edad')

c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == "":
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packa

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
Secuencia,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0202,ACACIAS,25,50-59,Española,M,1,Primera Atención,10.0,Malos Tratos,02/01/2018
2,83,CSS Zaida,11,CARABANCHEL,10,LATINA,1004,ALUCHE,82,50-59,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2018
3,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0102,EMBAJADORES,26,>=65,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2018
4,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0101,PALACIO,21,>=65,Española,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2018
5,74,CSS Santa Hortensia,5,CHAMARTÍN,5,CHAMARTÍN,0501,EL VISO,5,>=65,Española,H,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5627,98,CSS Pablo Casals,20,SAN BLAS-CANILLEJAS,20,SAN BLAS-CANILLEJAS,2004,ARCOS,48,30-39,Española,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,28/12/2018
5628,85,CSS Loyola de Palacio,12,USERA,12,USERA,1203,SAN FERMIN,100,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,28/12/2018
5629,84,CSS San Filiberto,12,USERA,12,USERA,1205,MOSCARDO,53,30-39,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,28/12/2018


In [12]:
'''Verificamos que ya no hay valores nulos'''
print(df_2018['Tramo Edad'].value_counts())
print("Los valores nulos que hay actualmente en este DataFrame son: ",df_2018['Tramo Edad'].isnull().sum())

print(df_2018['Nacionalidad'].value_counts())
print("Los valores nulos que hay en la columna de Nacionalidad son: ",df_2018['Nacionalidad'].isnull().sum())

print(df_2018['Sexo'].value_counts())
print("Los valores nulos que hay en la columna de Sexo son: ",df_2018['Sexo'].isnull().sum())

>=65     1423
>= 85    1194
30-39     866
40-49     786
50-59     558
19-29     555
60-64     214
<=18       35
Name: Tramo Edad, dtype: int64
Los valores nulos que hay actualmente en este DataFrame son:  0
Española      3782
Extranjera    1849
Name: Nacionalidad, dtype: int64
Los valores nulos que hay en la columna de Nacionalidad son:  0
M    3767
H    1864
Name: Sexo, dtype: int64
Los valores nulos que hay en la columna de Sexo son:  0


Hacemos un *mapeo* creando un diccionario y agregando una nueva columna **'Age_range_code'**. 

In [13]:
lista_rango_edad=df_2018['Tramo Edad'].unique()                  # Hay que revisar esto, porque no está ordenado y sería lo ideal
lista_rango_edad.sort()
mivalor = [ x for x in range(len(lista_rango_edad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
mivalor
lista_rango_edad=list(lista_rango_edad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 

Age_range_code = {k: v for k, v in zip(lista_rango_edad, mivalor)}   ##  sustituirlo por cero
Age_range_code
df_2018['Age_range_code']= df_2018['Tramo Edad'].map(Age_range_code)

Los valores son: {19-29: 0, 30-39: 1, 40-49: 2, 50-59: 3, 60-64: 4, <=18: 5, >=65: 6}

Hacemos otros dos mapeos, en este caso para __nacionalidad__ y __sexo__ del usuario, siendo los valores: 

0 para Española, 1 para Extranjera

0 para H, 1 para M

Durante este proceso buscaremos espacios en blanco y si los hay sustituiremos por el anterior.


In [14]:
lista_nacionalidad=df_2018['Nacionalidad'].unique()
lista_nacionalidad.sort()
lista_nacionalidad
mivalor = [ x for x in range(len(lista_nacionalidad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_nacionalidad=list(lista_nacionalidad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Nationality_code = {k: v for k, v in zip(lista_nacionalidad, mivalor)}   ##  sustituirlo por cero
df_2018['Nationality_code']=df_2018['Nacionalidad'].map(Nationality_code)
df_2018.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code
Secuencia,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0202,ACACIAS,25,50-59,Española,M,1,Primera Atención,10.0,Malos Tratos,02/01/2018,3,0
2,83,CSS Zaida,11,CARABANCHEL,10,LATINA,1004,ALUCHE,82,50-59,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2018,3,1
3,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0102,EMBAJADORES,26,>=65,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2018,7,0
4,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0101,PALACIO,21,>=65,Española,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2018,7,0
5,74,CSS Santa Hortensia,5,CHAMARTÍN,5,CHAMARTÍN,0501,EL VISO,5,>=65,Española,H,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2018,7,0


In [15]:
lista_sexo=df_2018['Sexo'].unique()
lista_sexo.sort()
lista_sexo
mivalor = [ x for x in range(len(lista_sexo))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_sexo=list(lista_sexo)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Sex_code = {k: v for k, v in zip(lista_sexo, mivalor)}   ##  sustituirlo por cero
df_2018['Sexo_code']=df_2018['Sexo'].map(Sex_code)
df_2018.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sexo_code
Secuencia,,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0202,ACACIAS,25,50-59,Española,M,1,Primera Atención,10.0,Malos Tratos,02/01/2018,3,0,1
2,83,CSS Zaida,11,CARABANCHEL,10,LATINA,1004,ALUCHE,82,50-59,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2018,3,1,1
3,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0102,EMBAJADORES,26,>=65,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2018,7,0,1
4,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0101,PALACIO,21,>=65,Española,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2018,7,0,1
5,74,CSS Santa Hortensia,5,CHAMARTÍN,5,CHAMARTÍN,0501,EL VISO,5,>=65,Española,H,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2018,7,0,0


In [16]:
print("Los valores NaN que hay en este Dataset son: ",df_2018['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2018))
df_2018['Código Tipo Supuesto Urgente'].value_counts()

Los valores NaN que hay en este Dataset son:  318
Los registros que hay en este DataSet son:  5631


40.0    2297
30.0    2023
50.0     712
10.0     251
20.0      30
Name: Código Tipo Supuesto Urgente, dtype: int64

En este Dataframe vemos que la media de **'Código Tipo Supuesto Urgente'** se aproxima a la mediana.

Este tipo de dato, aunque es numérico, es categorico y solo tienen valores de 10,20,30,40 y 50.

Hay 318 datos vacíos, los rellenaremos con los datos del registro anterior.

In [17]:
df_2018['Código Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
df_2018['Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
'''Verificamos'''
print("Los valores NaN que hay en este Dataset son: ",df_2018['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2018))
print("Los valores NaN que hay en este Dataset son: ",df_2018['Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2018))
df_2018.groupby('Código Tipo Supuesto Urgente')['Tipo Supuesto Urgente'].value_counts()

Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  5631
Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  5631


Código Tipo Supuesto Urgente  Tipo Supuesto Urgente              
10.0                          Malos Tratos                            269
20.0                          Abandono                                 31
30.0                          Alojamiento/Vivienda                   2129
40.0                          Desproteccion social problema salud    2432
50.0                          Desprotección socio-familiar            770
Name: Tipo Supuesto Urgente, dtype: int64

Vamos a dar un formato de fecha año y trimestre a los valores de __'Fecha Demanda'__ para poder ver qué evolución tienen los datos en el tiempo.

In [18]:
df_2018['Fecha Cita'] = pd.to_datetime(df_2018['Fecha Cita'], dayfirst=True)

df_2018['year'] = df_2018['Fecha Cita'].dt.year
df_2018['Quarteryear'] = df_2018['Fecha Cita'].dt.quarter
df_2018.head()


,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,...,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
Secuencia,,,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0202,ACACIAS,25,50-59,...,1,Primera Atención,10.0,Malos Tratos,2018-01-02,3,0,1,2018,1
2,83,CSS Zaida,11,CARABANCHEL,10,LATINA,1004,ALUCHE,82,50-59,...,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,2018-01-02,3,1,1,2018,1
3,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0102,EMBAJADORES,26,>=65,...,1,Primera Atención,30.0,Alojamiento/Vivienda,2018-01-02,7,0,1,2018,1
4,70,CSS Puerta de Toledo,1,CENTRO,1,CENTRO,0101,PALACIO,21,>=65,...,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,2018-01-02,7,0,1,2018,1
5,74,CSS Santa Hortensia,5,CHAMARTÍN,5,CHAMARTÍN,0501,EL VISO,5,>=65,...,2,Zona / Historia Social,40.0,Desproteccion social problema salud,2018-01-02,7,0,0,2018,1


In [19]:
col = df_2018.pop('Código Tipo Supuesto Urgente')
df_2018.insert(loc= 0 , column= 'Código Tipo Supuesto Urgente', value= col)
df_2018.drop(columns=['Código Centro','Centro','Código Distrito','Distrito Centro','Distrito','Tipo Atención','Código Barrio','Barrio','Sección Censal','Tramo Edad','Nacionalidad','Sexo','Código Tipo Atención','Tipo Supuesto Urgente','Fecha Cita'],inplace=True)
df_2018.head()

,Código Tipo Supuesto Urgente,Código Distrito Centro,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
Secuencia,,,,,,,
1,10.0,2,3,0,1,2018,1
2,30.0,11,3,1,1,2018,1
3,30.0,1,7,0,1,2018,1
4,30.0,1,7,0,1,2018,1
5,40.0,5,7,0,0,2018,1


#### 2019

In [20]:
df_2019['Tramo Edad'].value_counts()

>= 85      1037
80 - 84     481
40 - 44     432
35 - 39     402
30 - 34     386
45- 49      351
50 - 54     341
75 - 79     340
25 - 29     299
70 - 74     262
18 - 24     259
55 - 59     231
60 - 64     222
65 - 69     208
12 - 17      19
             14
4 - 11       10
0 - 3         4
Name: Tramo Edad, dtype: int64

In [22]:
df_2019['Tramo Edad']=df_2019['Tramo Edad'].mask(df_2019['Tramo Edad'] == ' ').fillna(method='ffill')#,inplace=True)
df_2019['Sexo']=df_2019['Sexo'].mask(df_2019['Sexo'] == ' ').fillna(method='ffill')
df_2019['Nacionalidad']=df_2019['Nacionalidad'].mask(df_2019['Nacionalidad'] == ' ').fillna(method='ffill')
regex_replacement(df_2019, 'Tramo Edad')

c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == "":
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packa

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
Secuencia,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0205,DELICIAS,98,40-49,Española,H,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2019
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0206,PALOS DE MOGUER,68,>=65,Española,H,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2019
3,82,CSS Monseñor Óscar Romero,11,CARABANCHEL,11,CARABANCHEL,1106,BUENAVISTA,155,>=65,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2019
4,82,CSS Monseñor Óscar Romero,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,97,30-39,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2019
5,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,79,>=65,Española,H,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5294,85,CSS Loyola de Palacio,12,USERA,12,USERA,1202,ORCASUR,21,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,30/12/2019
5295,84,CSS San Filiberto,12,USERA,12,USERA,1205,MOSCARDO,53,>=65,Española,H,1,Primera Atención,50.0,Desprotección socio-familiar,30/12/2019
5296,94,CSS Villa de Vallecas,18,VILLA DE VALLECAS,18,VILLA DE VALLECAS,1801,CASCO HISTORICO DE VALLECAS,4,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,30/12/2019


In [23]:
'''Verificamos que ya no hay valores nulos'''
print(df_2019['Tramo Edad'].value_counts())
print("Los valores nulos que hay actualmente en este DataFrame son: ",df_2019['Tramo Edad'].isnull().sum())

print(df_2019['Nacionalidad'].value_counts())
print("Los valores nulos que hay en la columna de Nacionalidad son: ",df_2019['Nacionalidad'].isnull().sum())

print(df_2019['Sexo'].value_counts())
print("Los valores nulos que hay en la columna de Sexo son: ",df_2019['Sexo'].isnull().sum())

>=65     1293
>= 85    1041
30-39     789
40-49     784
50-59     574
19-29     561
60-64     223
<=18       33
Name: Tramo Edad, dtype: int64
Los valores nulos que hay actualmente en este DataFrame son:  0
Española      3365
Extranjera    1933
Name: Nacionalidad, dtype: int64
Los valores nulos que hay en la columna de Nacionalidad son:  0
M    3682
H    1616
Name: Sexo, dtype: int64
Los valores nulos que hay en la columna de Sexo son:  0


In [24]:
lista_rango_edad=df_2019['Tramo Edad'].unique()                  # Hay que revisar esto, porque no está ordenado y sería lo ideal
lista_rango_edad.sort()
mivalor = [ x for x in range(len(lista_rango_edad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
mivalor
lista_rango_edad=list(lista_rango_edad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 

Age_range_code = {k: v for k, v in zip(lista_rango_edad, mivalor)}   ##  sustituirlo por cero
Age_range_code
df_2019['Age_range_code']= df_2019['Tramo Edad'].map(Age_range_code)

In [25]:
lista_nacionalidad=df_2019['Nacionalidad'].unique()
lista_nacionalidad.sort()
lista_nacionalidad
mivalor = [ x for x in range(len(lista_nacionalidad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_nacionalidad=list(lista_nacionalidad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Nationality_code = {k: v for k, v in zip(lista_nacionalidad, mivalor)}   ##  sustituirlo por cero
df_2019['Nationality_code']=df_2019['Nacionalidad'].map(Nationality_code)
df_2019.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code
Secuencia,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0205,DELICIAS,98,40-49,Española,H,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2019,2,0
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0206,PALOS DE MOGUER,68,>=65,Española,H,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2019,7,0
3,82,CSS Monseñor Óscar Romero,11,CARABANCHEL,11,CARABANCHEL,1106,BUENAVISTA,155,>=65,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2019,7,0
4,82,CSS Monseñor Óscar Romero,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,97,30-39,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2019,1,1
5,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,79,>=65,Española,H,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2019,7,0


In [26]:
lista_sexo=df_2019['Sexo'].unique()
lista_sexo.sort()
lista_sexo
mivalor = [ x for x in range(len(lista_sexo))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_sexo=list(lista_sexo)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Sex_code = {k: v for k, v in zip(lista_sexo, mivalor)}   ##  sustituirlo por cero
df_2019['Sexo_code']=df_2019['Sexo'].map(Sex_code)
df_2019.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sexo_code
Secuencia,,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0205,DELICIAS,98,40-49,Española,H,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2019,2,0,0
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0206,PALOS DE MOGUER,68,>=65,Española,H,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2019,7,0,0
3,82,CSS Monseñor Óscar Romero,11,CARABANCHEL,11,CARABANCHEL,1106,BUENAVISTA,155,>=65,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2019,7,0,1
4,82,CSS Monseñor Óscar Romero,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,97,30-39,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2019,1,1,1
5,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,79,>=65,Española,H,1,Primera Atención,30.0,Alojamiento/Vivienda,02/01/2019,7,0,0


In [27]:
print("Los valores NaN que hay en este Dataset son: ",df_2019['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2019))
df_2019['Código Tipo Supuesto Urgente'].value_counts()

Los valores NaN que hay en este Dataset son:  298
Los registros que hay en este DataSet son:  5298


40.0    2128
30.0    2020
50.0     588
10.0     248
20.0      16
Name: Código Tipo Supuesto Urgente, dtype: int64

En este Dataframe vemos que la mediana de **'Código Tipo Supuesto Urgente'** tiene muchos valores en 40 y la media está entre 30 y 40 también, lo que indica que en general casi todos los registros se situan entre 30 y 40.

Este tipo de dato, aunque es numérico, es categorico y solo tienen valores de 10,20,30,40 y 50.

Hay 298 datos vacíos, los rellenaremos con los datos del registro anterior, que principalmente serán éstos.

In [28]:
df_2019['Código Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
df_2019['Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
'''Verificamos'''
print("Los valores NaN que hay en este Dataset son: ",df_2019['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2019))
print("Los valores NaN que hay en este Dataset son: ",df_2019['Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2019))
df_2019.groupby('Código Tipo Supuesto Urgente')['Tipo Supuesto Urgente'].value_counts()

Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  5298
Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  5298


Código Tipo Supuesto Urgente  Tipo Supuesto Urgente              
10.0                          Malos Tratos                            264
20.0                          Abandono                                 19
30.0                          Alojamiento/Vivienda                   2149
40.0                          Desproteccion social problema salud    2244
50.0                          Desprotección socio-familiar            622
Name: Tipo Supuesto Urgente, dtype: int64

In [29]:
df_2019['Fecha Cita'] = pd.to_datetime(df_2019['Fecha Cita'], dayfirst=True)

df_2019['year'] = df_2019['Fecha Cita'].dt.year
df_2019['Quarteryear'] = df_2019['Fecha Cita'].dt.quarter
df_2019.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,...,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
Secuencia,,,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0205,DELICIAS,98,40-49,...,1,Primera Atención,30.0,Alojamiento/Vivienda,2019-01-02,2,0,0,2019,1
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,0206,PALOS DE MOGUER,68,>=65,...,2,Zona / Historia Social,40.0,Desproteccion social problema salud,2019-01-02,7,0,0,2019,1
3,82,CSS Monseñor Óscar Romero,11,CARABANCHEL,11,CARABANCHEL,1106,BUENAVISTA,155,>=65,...,2,Zona / Historia Social,40.0,Desproteccion social problema salud,2019-01-02,7,0,1,2019,1
4,82,CSS Monseñor Óscar Romero,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,97,30-39,...,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,2019-01-02,1,1,1,2019,1
5,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,79,>=65,...,1,Primera Atención,30.0,Alojamiento/Vivienda,2019-01-02,7,0,0,2019,1


In [30]:
col = df_2019.pop('Código Tipo Supuesto Urgente')
df_2019.insert(loc= 0 , column= 'Código Tipo Supuesto Urgente', value= col)
df_2019.drop(columns=['Código Centro','Centro','Código Distrito','Distrito Centro','Distrito','Tipo Atención','Código Barrio','Barrio','Sección Censal','Tramo Edad','Nacionalidad','Sexo','Código Tipo Atención','Tipo Supuesto Urgente','Fecha Cita'],inplace=True)
df_2019.head()

,Código Tipo Supuesto Urgente,Código Distrito Centro,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
Secuencia,,,,,,,
1,30.0,2,2,0,0,2019,1
2,40.0,2,7,0,0,2019,1
3,40.0,11,7,0,1,2019,1
4,30.0,11,1,1,1,2019,1
5,30.0,11,7,0,0,2019,1


#### 2020

In [31]:
df_2020.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
Secuencia,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,50,50 - 54,Española,H,1,Primera Atención,50.0,Desprotección socio-familiar,02/01/2020
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,62,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2020
3,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,201,IMPERIAL,90,60 - 64,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2020
4,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,206,PALOS DE MOGUER,64,75 - 79,Española,H,1,Primera Atención,40.0,Desproteccion social problema salud,02/01/2020
5,99,CSS Teresa de Calcuta,21,BARAJAS,21,BARAJAS,2101,ALAMEDA DE OSUNA,11,40 - 44,Española,M,2,Zona / Historia Social,50.0,Desprotección socio-familiar,02/01/2020


In [32]:
df_2020['Tramo Edad'].value_counts()

40 - 44    3070
35 - 39    3035
30 - 34    2619
>= 85      2501
45- 49     2287
25 - 29    1901
50 - 54    1898
55 - 59    1329
80 - 84    1178
18 - 24    1154
60 - 64    1045
75 - 79     800
70 - 74     698
65 - 69     673
12 - 17      66
             44
4 - 11       41
0 - 3        12
Name: Tramo Edad, dtype: int64

In [33]:
df_2020['Tramo Edad']=df_2020['Tramo Edad'].mask(df_2020['Tramo Edad'] == ' ').fillna(method='ffill')#,inplace=True)
df_2020['Sexo']=df_2020['Sexo'].mask(df_2020['Sexo'] == ' ').fillna(method='ffill')
df_2020['Nacionalidad']=df_2020['Nacionalidad'].mask(df_2020['Nacionalidad'] == ' ').fillna(method='ffill')
regex_replacement(df_2020, 'Tramo Edad')

c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == "":
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packa

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
Secuencia,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,50,50-59,Española,H,1,Primera Atención,50.0,Desprotección socio-familiar,02/01/2020
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,62,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2020
3,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,201,IMPERIAL,90,60-64,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2020
4,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,206,PALOS DE MOGUER,64,>=65,Española,H,1,Primera Atención,40.0,Desproteccion social problema salud,02/01/2020
5,99,CSS Teresa de Calcuta,21,BARAJAS,21,BARAJAS,2101,ALAMEDA DE OSUNA,11,40-49,Española,M,2,Zona / Historia Social,50.0,Desprotección socio-familiar,02/01/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24347,130,CSS El Querol,16,HORTALEZA,16,HORTALEZA,1605,APOSTOL SANTIAGO,82,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,30/12/2020
24348,86,CSS San Diego,13,PUENTE DE VALLECAS,13,PUENTE DE VALLECAS,1303,PALOMERAS BAJAS,83,19-29,Extranjera,M,1,Primera Atención,30.0,Alojamiento/Vivienda,30/12/2020
24349,86,CSS San Diego,13,PUENTE DE VALLECAS,13,PUENTE DE VALLECAS,1303,PALOMERAS BAJAS,104,>= 85,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,30/12/2020


In [34]:
'''Verificamos que ya no hay valores nulos'''
print(df_2020['Tramo Edad'].value_counts())
print("Los valores nulos que hay actualmente en este DataFrame son: ",df_2020['Tramo Edad'].isnull().sum())

print(df_2020['Nacionalidad'].value_counts())
print("Los valores nulos que hay en la columna de Nacionalidad son: ",df_2020['Nacionalidad'].isnull().sum())

print(df_2020['Sexo'].value_counts())
print("Los valores nulos que hay en la columna de Sexo son: ",df_2020['Sexo'].isnull().sum())

30-39    5666
40-49    5367
>=65     3353
50-59    3231
19-29    3061
>= 85    2507
60-64    1047
<=18      119
Name: Tramo Edad, dtype: int64
Los valores nulos que hay actualmente en este DataFrame son:  0
Extranjera    14944
Española       9407
Name: Nacionalidad, dtype: int64
Los valores nulos que hay en la columna de Nacionalidad son:  0
M    17398
H     6953
Name: Sexo, dtype: int64
Los valores nulos que hay en la columna de Sexo son:  0


In [35]:
lista_rango_edad=df_2020['Tramo Edad'].unique()                  # Hay que revisar esto, porque no está ordenado y sería lo ideal
lista_rango_edad.sort()
mivalor = [ x for x in range(len(lista_rango_edad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
mivalor
lista_rango_edad=list(lista_rango_edad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 

Age_range_code = {k: v for k, v in zip(lista_rango_edad, mivalor)}   ##  sustituirlo por cero
Age_range_code
df_2020['Age_range_code']= df_2020['Tramo Edad'].map(Age_range_code)

In [36]:
lista_nacionalidad=df_2020['Nacionalidad'].unique()
lista_nacionalidad.sort()
lista_nacionalidad
mivalor = [ x for x in range(len(lista_nacionalidad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_nacionalidad=list(lista_nacionalidad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Nationality_code = {k: v for k, v in zip(lista_nacionalidad, mivalor)}   ##  sustituirlo por cero
df_2020['Nationality_code']=df_2020['Nacionalidad'].map(Nationality_code)
df_2020.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code
Secuencia,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,50,50-59,Española,H,1,Primera Atención,50.0,Desprotección socio-familiar,02/01/2020,3,0
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,62,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2020,6,0
3,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,201,IMPERIAL,90,60-64,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2020,4,1
4,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,206,PALOS DE MOGUER,64,>=65,Española,H,1,Primera Atención,40.0,Desproteccion social problema salud,02/01/2020,7,0
5,99,CSS Teresa de Calcuta,21,BARAJAS,21,BARAJAS,2101,ALAMEDA DE OSUNA,11,40-49,Española,M,2,Zona / Historia Social,50.0,Desprotección socio-familiar,02/01/2020,2,0


In [37]:
lista_sexo=df_2020['Sexo'].unique()
lista_sexo.sort()
lista_sexo
mivalor = [ x for x in range(len(lista_sexo))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_sexo=list(lista_sexo)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Sex_code = {k: v for k, v in zip(lista_sexo, mivalor)}   ##  sustituirlo por cero
df_2020['Sexo_code']=df_2020['Sexo'].map(Sex_code)
df_2020.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sexo_code
Secuencia,,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,50,50-59,Española,H,1,Primera Atención,50.0,Desprotección socio-familiar,02/01/2020,3,0,0
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,62,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2020,6,0,1
3,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,201,IMPERIAL,90,60-64,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2020,4,1,0
4,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,206,PALOS DE MOGUER,64,>=65,Española,H,1,Primera Atención,40.0,Desproteccion social problema salud,02/01/2020,7,0,0
5,99,CSS Teresa de Calcuta,21,BARAJAS,21,BARAJAS,2101,ALAMEDA DE OSUNA,11,40-49,Española,M,2,Zona / Historia Social,50.0,Desprotección socio-familiar,02/01/2020,2,0,1


In [38]:
print("Los valores NaN que hay en este Dataset son: ",df_2020['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2020))
df_2020['Código Tipo Supuesto Urgente'].value_counts()

Los valores NaN que hay en este Dataset son:  1402
Los registros que hay en este DataSet son:  24351


50.0    16475
40.0     4308
30.0     1875
10.0      268
20.0       23
Name: Código Tipo Supuesto Urgente, dtype: int64

In [39]:
df_2020['Código Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
df_2020['Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
'''Verificamos'''
print("Los valores NaN que hay en este Dataset son: ",df_2020['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2020))
print("Los valores NaN que hay en este Dataset son: ",df_2020['Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2020))
df_2020.groupby('Código Tipo Supuesto Urgente')['Tipo Supuesto Urgente'].value_counts()

Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  24351
Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  24351


Código Tipo Supuesto Urgente  Tipo Supuesto Urgente              
10.0                          Malos Tratos                             290
20.0                          Abandono                                  23
30.0                          Alojamiento/Vivienda                    2016
40.0                          Desproteccion social problema salud     4539
50.0                          Desprotección socio-familiar           17483
Name: Tipo Supuesto Urgente, dtype: int64

In [40]:
df_2020['Fecha Cita'] = pd.to_datetime(df_2020['Fecha Cita'], dayfirst=True)

df_2020['year'] = df_2020['Fecha Cita'].dt.year
df_2020['Quarteryear'] = df_2020['Fecha Cita'].dt.quarter
df_2020.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,...,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
Secuencia,,,,,,,,,,,,,,,,,,,,,
1,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,50,50-59,...,1,Primera Atención,50.0,Desprotección socio-familiar,2020-01-02,3,0,0,2020,1
2,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,205,DELICIAS,62,>= 85,...,2,Zona / Historia Social,40.0,Desproteccion social problema salud,2020-01-02,6,0,1,2020,1
3,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,201,IMPERIAL,90,60-64,...,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,2020-01-02,4,1,0,2020,1
4,69,CSS José Villarreal,2,ARGANZUELA,2,ARGANZUELA,206,PALOS DE MOGUER,64,>=65,...,1,Primera Atención,40.0,Desproteccion social problema salud,2020-01-02,7,0,0,2020,1
5,99,CSS Teresa de Calcuta,21,BARAJAS,21,BARAJAS,2101,ALAMEDA DE OSUNA,11,40-49,...,2,Zona / Historia Social,50.0,Desprotección socio-familiar,2020-01-02,2,0,1,2020,1


In [41]:
col = df_2020.pop('Código Tipo Supuesto Urgente')
df_2020.insert(loc= 0 , column= 'Código Tipo Supuesto Urgente', value= col)
df_2020.drop(columns=['Código Centro','Centro','Distrito Centro','Código Distrito','Distrito','Tipo Atención','Código Barrio','Barrio','Sección Censal','Tramo Edad','Nacionalidad','Sexo','Código Tipo Atención','Tipo Supuesto Urgente','Fecha Cita'],inplace=True)
df_2020.head()

,Código Tipo Supuesto Urgente,Código Distrito Centro,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
Secuencia,,,,,,,
1,50.0,2,3,0,0,2020,1
2,40.0,2,6,0,1,2020,1
3,30.0,2,4,1,0,2020,1
4,40.0,2,7,0,0,2020,1
5,50.0,21,2,0,1,2020,1


#### 2021

In [42]:
df_2021.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
Secuencia,,,,,,,,,,,,,,,,,
1,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1103,SAN ISIDRO,060,25 - 29,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021
2,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,083,30 - 34,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,04/01/2021
3,74,CSS Santa Hortensia,5,CHAMARTÍN,05,CHAMARTÍN,0503,CIUDAD JARDIN,043,70 - 74,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021
4,91,CSS Luis Vives,15,CIUDAD LINEAL,15,CIUDAD LINEAL,1503,QUINTANA,098,25 - 29,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021
5,80,CSS Gallur,10,LATINA,10,LATINA,1003,LUCERO,068,>= 85,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021


In [100]:
df_2021['Tramo Edad'].value_counts()

>= 85      473
40 - 44    221
35 - 39    205
80 - 84    189
75 - 79    186
50 - 54    181
45- 49     168
30 - 34    163
60 - 64    153
55 - 59    148
70 - 74    145
25 - 29    140
18 - 24    113
65 - 69    112
12 - 17     18
4 - 11       4
             4
0 - 3        3
Name: Tramo Edad, dtype: int64

In [43]:
df_2021['Tramo Edad']=df_2021['Tramo Edad'].mask(df_2021['Tramo Edad'] == ' ').fillna(method='ffill')#,inplace=True)
df_2021['Sexo']=df_2021['Sexo'].mask(df_2021['Sexo'] == ' ').fillna(method='ffill')
df_2021['Nacionalidad']=df_2021['Nacionalidad'].mask(df_2021['Nacionalidad'] == ' ').fillna(method='ffill')
regex_replacement(df_2021, 'Tramo Edad')

c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == "":
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packa

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
Secuencia,,,,,,,,,,,,,,,,,
1,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1103,SAN ISIDRO,060,19-29,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021
2,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,083,30-39,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,04/01/2021
3,74,CSS Santa Hortensia,5,CHAMARTÍN,05,CHAMARTÍN,0503,CIUDAD JARDIN,043,>=65,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021
4,91,CSS Luis Vives,15,CIUDAD LINEAL,15,CIUDAD LINEAL,1503,QUINTANA,098,19-29,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021
5,80,CSS Gallur,10,LATINA,10,LATINA,1003,LUCERO,068,>= 85,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,92,CSS Concepción Arenal,16,HORTALEZA,16,HORTALEZA,1604,PINAR DEL REY,044,>= 85,Española,H,1,Primera Atención,40.0,Desproteccion social problema salud,12/07/2021
2623,77,CSS Badalona,8,FUENCARRAL-EL PARDO,08,FUENCARRAL-EL PARDO,0802,FUENTELARREINA,006,>= 85,Española,H,1,Primera Atención,50.0,Desprotección socio-familiar,14/07/2021
2624,77,CSS Badalona,8,FUENCARRAL-EL PARDO,08,FUENCARRAL-EL PARDO,0806,VALVERDE,113,>=65,Española,H,2,Zona / Historia Social,50.0,Desprotección socio-familiar,14/07/2021


In [44]:
'''Verificamos que ya no hay valores nulos'''
print(df_2021['Tramo Edad'].value_counts())
print("Los valores nulos que hay actualmente en este DataFrame son: ",df_2021['Tramo Edad'].isnull().sum())

print(df_2021['Nacionalidad'].value_counts())
print("Los valores nulos que hay en la columna de Nacionalidad son: ",df_2021['Nacionalidad'].isnull().sum())

print(df_2021['Sexo'].value_counts())
print("Los valores nulos que hay en la columna de Sexo son: ",df_2021['Sexo'].isnull().sum())

>=65     634
>= 85    473
40-49    389
30-39    370
50-59    329
19-29    253
60-64    153
<=18      25
Name: Tramo Edad, dtype: int64
Los valores nulos que hay actualmente en este DataFrame son:  0
Española      1697
Extranjera     929
Name: Nacionalidad, dtype: int64
Los valores nulos que hay en la columna de Nacionalidad son:  0
M    1719
H     907
Name: Sexo, dtype: int64
Los valores nulos que hay en la columna de Sexo son:  0


In [45]:
lista_rango_edad=df_2021['Tramo Edad'].unique()                  # Hay que revisar esto, porque no está ordenado y sería lo ideal
lista_rango_edad.sort()
mivalor = [ x for x in range(len(lista_rango_edad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
mivalor
lista_rango_edad=list(lista_rango_edad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 

Age_range_code = {k: v for k, v in zip(lista_rango_edad, mivalor)}   ##  sustituirlo por cero
Age_range_code
df_2021['Age_range_code']= df_2021['Tramo Edad'].map(Age_range_code)

In [47]:
lista_nacionalidad=df_2021['Nacionalidad'].unique()
lista_nacionalidad.sort()
lista_nacionalidad
mivalor = [ x for x in range(len(lista_nacionalidad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_nacionalidad=list(lista_nacionalidad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Nationality_code = {k: v for k, v in zip(lista_nacionalidad, mivalor)}   ##  sustituirlo por cero
df_2021['Nationality_code']=df_2021['Nacionalidad'].map(Nationality_code)
df_2021.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code
Secuencia,,,,,,,,,,,,,,,,,,,
1,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1103,SAN ISIDRO,060,19-29,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021,0,1
2,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,083,30-39,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,04/01/2021,1,0
3,74,CSS Santa Hortensia,5,CHAMARTÍN,05,CHAMARTÍN,0503,CIUDAD JARDIN,043,>=65,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021,7,0
4,91,CSS Luis Vives,15,CIUDAD LINEAL,15,CIUDAD LINEAL,1503,QUINTANA,098,19-29,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021,0,1
5,80,CSS Gallur,10,LATINA,10,LATINA,1003,LUCERO,068,>= 85,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021,6,0


In [48]:
lista_sexo=df_2021['Sexo'].unique()
lista_sexo.sort()
lista_sexo
mivalor = [ x for x in range(len(lista_sexo))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_sexo=list(lista_sexo)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Sex_code = {k: v for k, v in zip(lista_sexo, mivalor)}   ##  sustituirlo por cero
df_2021['Sexo_code']=df_2021['Sexo'].map(Sex_code)
df_2021.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sexo_code
Secuencia,,,,,,,,,,,,,,,,,,,,
1,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1103,SAN ISIDRO,060,19-29,Extranjera,M,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021,0,1,1
2,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,083,30-39,Española,M,1,Primera Atención,30.0,Alojamiento/Vivienda,04/01/2021,1,0,1
3,74,CSS Santa Hortensia,5,CHAMARTÍN,05,CHAMARTÍN,0503,CIUDAD JARDIN,043,>=65,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021,7,0,1
4,91,CSS Luis Vives,15,CIUDAD LINEAL,15,CIUDAD LINEAL,1503,QUINTANA,098,19-29,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,04/01/2021,0,1,0
5,80,CSS Gallur,10,LATINA,10,LATINA,1003,LUCERO,068,>= 85,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,04/01/2021,6,0,1


In [49]:
print("Los valores NaN que hay en este Dataset son: ",df_2021['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2021))
df_2021['Código Tipo Supuesto Urgente'].value_counts() 

Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  2626


40.0    1133
30.0    1037
50.0     312
10.0     134
20.0      10
Name: Código Tipo Supuesto Urgente, dtype: int64

In [111]:
df_2021['Tipo Supuesto Urgente'].value_counts()

Desproteccion social problema salud    1133
Alojamiento/Vivienda                   1037
Desprotección socio-familiar            312
Malos Tratos                            134
Abandono                                 10
Name: Tipo Supuesto Urgente, dtype: int64

In [ ]:
df_2021['Código Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
df_2021['Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
'''Verificamos'''
print("Los valores NaN que hay en este Dataset son: ",df_2021['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2021))
print("Los valores NaN que hay en este Dataset son: ",df_2021['Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2021))
df_2021.groupby('Código Tipo Supuesto Urgente')['Tipo Supuesto Urgente'].value_counts()

In [50]:
df_2021['Fecha Cita'] = pd.to_datetime(df_2021['Fecha Cita'], dayfirst=True)

df_2021['year'] = df_2021['Fecha Cita'].dt.year
df_2021['Quarteryear'] = df_2021['Fecha Cita'].dt.quarter
df_2021.head()

,Código Centro,Centro,Código Distrito Centro,Distrito Centro,Código Distrito,Distrito,Código Barrio,Barrio,Sección Censal,Tramo Edad,...,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
Secuencia,,,,,,,,,,,,,,,,,,,,,
1,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1103,SAN ISIDRO,060,19-29,...,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,2021-01-04,0,1,1,2021,1
2,83,CSS Zaida,11,CARABANCHEL,11,CARABANCHEL,1104,VISTA ALEGRE,083,30-39,...,1,Primera Atención,30.0,Alojamiento/Vivienda,2021-01-04,1,0,1,2021,1
3,74,CSS Santa Hortensia,5,CHAMARTÍN,05,CHAMARTÍN,0503,CIUDAD JARDIN,043,>=65,...,1,Primera Atención,40.0,Desproteccion social problema salud,2021-01-04,7,0,1,2021,1
4,91,CSS Luis Vives,15,CIUDAD LINEAL,15,CIUDAD LINEAL,1503,QUINTANA,098,19-29,...,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,2021-01-04,0,1,0,2021,1
5,80,CSS Gallur,10,LATINA,10,LATINA,1003,LUCERO,068,>= 85,...,1,Primera Atención,40.0,Desproteccion social problema salud,2021-01-04,6,0,1,2021,1


In [51]:
col = df_2021.pop('Código Tipo Supuesto Urgente')
df_2021.insert(loc= 0 , column= 'Código Tipo Supuesto Urgente', value= col)
df_2021.drop(columns=['Código Centro','Centro','Código Distrito','Distrito Centro','Distrito','Tipo Atención','Código Barrio','Barrio','Sección Censal','Tramo Edad','Nacionalidad','Sexo','Código Tipo Atención','Tipo Supuesto Urgente','Fecha Cita'],inplace=True)
df_2021.head()

,Código Tipo Supuesto Urgente,Código Distrito Centro,Age_range_code,Nationality_code,Sexo_code,year,Quarteryear
Secuencia,,,,,,,
1,30.0,11,0,1,1,2021,1
2,30.0,11,1,0,1,2021,1
3,40.0,5,7,0,1,2021,1
4,30.0,15,0,1,0,2021,1
5,40.0,10,6,0,1,2021,1


Aqui vamos a generar varios fichero .csv para operar en los siguientes Notebooks.

- Vamos a generar un Dataframe completo que llamaremos __df_urgencias_total__.
- Vamos a generar dos DataFrames parciales ya que los datos del año 2020 están claramente desbalanceados y por ello vamos a dejar el año 2020 para realizar Ingenieria de Undersampling y el resto en otro DataFrame a la espera de la ingenieria.

In [52]:
df_urgencias_total=pd.concat([df_2021,df_2020,df_2019,df_2018], ignore_index=True)
df_urgencias_total.to_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_urgencias_total.csv',header=True,index=False)

In [53]:
df_urgencias_to_resample=pd.concat([df_2021,df_2019,df_2018], ignore_index=True)
df_urgencias_to_resample.to_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_urgencias_to_resample.csv',header=True,index=False)
df_2020.to_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_2020_to_resample.csv',header=True,index=False)
